## Your Turn -- Activity I:  Bacterial Growth 

In [2]:
print('\nEnabling interactive shell outputs ...')
print('   Use command pass; to disable cell text outputs')
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import warnings
warnings.filterwarnings('ignore') 
warnings.simplefilter(action="ignore",category=UserWarning)
warnings.simplefilter(action="ignore",category=FutureWarning)

import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import statsmodels.api as sm



%matplotlib inline
%config InlineBackend.figure_formats = {'png', 'retina'}


Enabling interactive shell outputs ...
   Use command pass; to disable cell text outputs


In [3]:
growth_df = pd.read_excel('lect09-2k-factorial.xlsx', sheet_name='Bacteria')
growth_df

,A,B,Rep1,Rep2,Rep3,Rep4,Rep5,Rep6
0,-1,-1,21,22,23,28,20,26
1,1,-1,37,39,38,38,35,36
2,-1,1,25,26,24,25,29,27
3,1,1,31,34,29,33,30,35


## Analyze the bacterial growth data in the table below and draw conclusions.

In [4]:
factorial_eff = pd.DataFrame(growth_df, columns=['A','B'])

# Add interaction effects
factorial_eff['AB'] = factorial_eff['A']*factorial_eff['B']

growth_y = np.array([[21,22,23,28,20,26],
             [37,39,38,38,35,36],
             [25,26,24,25,29,27],
             [31,34,29,33,30,35]])

# Get a vector of total response
total_growth = np.c_[growth_y.sum(axis=1)]
factorial_eff, total_growth

(   A  B  AB
 0 -1 -1   1
 1  1 -1  -1
 2 -1  1  -1
 3  1  1   1,
 array([[140],
        [223],
        [156],
        [192]]))

In [5]:
k, n = np.log2(len(factorial_eff)), growth_y.shape[1]

contrast_eff = factorial_eff.iloc[:,0:].mul(total_growth).sum()
print('Contrast effects\n', contrast_eff.to_string(), sep='')

effects = (contrast_eff)/((2**(k-1))*n)
print('\nEffect estimates\n', effects.to_string(), sep='')

ss_eff = (contrast_eff**2)/((2**k)*n)
print('\nSS effects\n', ss_eff.to_string(), sep='')

Contrast effects
A     119
B     -15
AB    -47

Effect estimates
A     9.916667
B    -1.250000
AB   -3.916667

SS effects
A     590.041667
B       9.375000
AB     92.041667


In [6]:
SST = (sum(sum(growth_y**2)) - sum(total_growth)**2/growth_y.size)[0]
SSE = SST - sum(ss_eff)
MSE = SSE/(growth_y.size - len(ss_eff) - 1)
print("SSE = {:.2f}, MSE = {:.2f}".format(SSE, MSE))

SSE = 102.17, MSE = 5.11


In [7]:
stats.f.sf(ss_eff['A']/MSE,1,20)

9.290524855305233e-10

In [8]:
stats.f.sf(ss_eff['B']/MSE,1,20)

0.19061715658088677

In [9]:
stats.f.sf(ss_eff['AB']/MSE,1,20)

0.0003969387187647818

### จากการทำ test statistic พบว่าค่า Time และค่า interaction ระหว่าง AB significant มีค่ามากกว่า 0.05 
### แต่ค่า Culture Medium not significant ค่า p-value < 0.05  

## Write down a regression model for predicting the bacterial growth using the effect estimates.

In [11]:
total_avg = sum(total_growth)/24
total_avg

array([29.625])

In [13]:
half_A = effects['A']/2
half_B = effects['B']/2
half_AB = effects['AB']/2
half_A, half_B, half_AB

(4.958333333333333, -0.625, -1.9583333333333333)